In [1]:
from tempfile import NamedTemporaryFile
from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt

import numpy as np

import lombardi.data
import lombardi.stats

from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=4, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)


def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [8]:
player = 'DeMarco Murray'
metric = 'rushing_yds'

game_range = 16

x_max = lombardi.data.metric_range(metric)

fig = plt.figure(figsize=(10, 5))
ax = plt.axes(xlim=(0,x_max), ylim=(0, .01))
line, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)

player_data = lombardi.data.player_metric(player, metric)

start = 0
end = len(player_data) - game_range
x = np.linspace(0, x_max, 1000)

def init():
    line.set_data([],[])
    line2.set_data([],[])
    return (line,line2) + (None,)

def animate(i):

    # running (last 16 games)
    data = player_data[i:i+game_range+1]
    x1, y1 = lombardi.stats.fit_gamma(x, data)
    line.set_data(x1,y1)

    # cumulative (all games)
    data2 = player_data[0:17 + i]
    x2,y2 = lombardi.stats.fit_gamma(x, data2)
    line2.set_data(x2,y2)

    return (line, line2) + (None,)

ani = animation.FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=range(end),
    interval=1000,
)
display_animation(ani)
